# "2019 April One Day Holding Long Short Portfolio BackTest"
> "2019 April Portfolio"
- toc: false
- branch: master
- badges: false
- comments: true
- categories: [long, short, longshort]
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

In [13]:
#hide_input
import numpy as np
import pandas as pd
import seaborn as sns
import os
import time
import gc
import glob
import matplotlib.pyplot as plt

In [14]:
#hide_input
df_testp=pd.read_csv('D:\\Downloads\\20194dayreturnpred2019train60K.csv')

In [15]:
#hide
df_testp=df_testp.loc[df_testp.Date>=20190401].loc[df_testp.Date<20190501].sort_values(by='Date',ascending=True).copy()

**Long-Short Portfolio**

In [16]:
#hide
df_agg=df_testp[['Date','pred','labelr1']].groupby(['Date','pred']).sum().reset_index();
df_agg['cumreturn1']=0.0
for l in range(len(df_agg)):
    df_agg['cumreturn1'][l]=np.prod((1+(df_agg['labelr1']/100))[:l])



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [17]:
#hide_input
df_agg

,Date,pred,labelr1,cumreturn1
0,20190401,-1.0,-32.898793,1.000000
1,20190401,1.0,11.760590,0.671012
2,20190402,1.0,0.787541,0.749927
3,20190403,-1.0,0.000000,0.755833
4,20190403,1.0,-19.842095,0.755833
5,20190404,-1.0,-8.333333,0.605860
6,20190404,1.0,24.063125,0.555372
7,20190405,1.0,3.301455,0.689011
8,20190408,-1.0,0.000000,0.711759
9,20190408,1.0,0.000000,0.711759


In [18]:
#hide_input
import altair as alt
import datetime
df_agg['Date']=df_agg['Date'].astype(str).str[:4]+'-'+df_agg['Date'].astype(str).str[4:6]+'-'+df_agg['Date'].astype(str).str[6:]
df_agg['Date']=df_agg['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
alt.Chart(df_agg).mark_trail().encode(
    x='Date:T',
    y='cumreturn1:Q',
    size='cumreturn1'
).properties(
    width=1000,
    height=1000
)

alt.Chart(...)

**Long portfolio**

In [19]:
#hide
df_agg=df_testp[['Date','pred','labelr1']].loc[df_testp.pred==1.0].groupby(['Date','pred']).sum().reset_index();
df_agg['cumreturn1']=0.0
for l in range(len(df_agg)):
    df_agg['cumreturn1'][l]=np.prod((1+(df_agg['labelr1']/100))[:l])



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [20]:
#hide_input
df_agg

,Date,pred,labelr1,cumreturn1
0,20190401,1.0,11.760590,1.000000
1,20190402,1.0,0.787541,1.117606
2,20190403,1.0,-19.842095,1.126408
3,20190404,1.0,24.063125,0.902905
4,20190405,1.0,3.301455,1.120172
5,20190408,1.0,0.000000,1.157154
6,20190409,1.0,0.000000,1.157154
7,20190410,1.0,28.794546,1.157154
8,20190411,1.0,-31.296296,1.490351
9,20190412,1.0,-28.240650,1.023926


In [21]:
#hide_input
import altair as alt
import datetime
df_agg['Date']=df_agg['Date'].astype(str).str[:4]+'-'+df_agg['Date'].astype(str).str[4:6]+'-'+df_agg['Date'].astype(str).str[6:]
df_agg['Date']=df_agg['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
alt.Chart(df_agg).mark_trail().encode(
    x='Date:T',
    y='cumreturn1:Q',
    size='cumreturn1'
).properties(
    width=1000,
    height=1000
)

alt.Chart(...)

**Short Portfolio**

In [22]:
#hide
df_agg=df_testp[['Date','pred','labelr1']].loc[df_testp.pred==-1.0].groupby(['Date','pred']).sum().reset_index();
df_agg['cumreturn1']=0.0
for l in range(len(df_agg)):
    df_agg['cumreturn1'][l]=np.prod((1+(df_agg['labelr1']/100))[:l])



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [23]:
#hide_input
df_agg

,Date,pred,labelr1,cumreturn1
0,20190401,-1.0,-32.898793,1.000000
1,20190403,-1.0,0.000000,0.671012
2,20190404,-1.0,-8.333333,0.671012
3,20190408,-1.0,0.000000,0.615094
4,20190415,-1.0,-0.449438,0.615094
5,20190416,-1.0,-4.869574,0.612330
6,20190418,-1.0,0.000000,0.582512
7,20190425,-1.0,0.000000,0.582512


In [24]:
#hide_input
import altair as alt
import datetime
df_agg['Date']=df_agg['Date'].astype(str).str[:4]+'-'+df_agg['Date'].astype(str).str[4:6]+'-'+df_agg['Date'].astype(str).str[6:]
df_agg['Date']=df_agg['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
alt.Chart(df_agg).mark_trail().encode(
    x='Date:T',
    y='cumreturn1:Q',
    size='cumreturn1'
).properties(
    width=1000,
    height=1000
)

alt.Chart(...)